# QUELLE RÉPARTITION DES DOCUMENTS ?

In [44]:
import pandas as pd
from datetime import datetime as dt
from datetime import date

from kiblib.utils.db import DbConn
from kiblib.document import Document

db_conn = DbConn().create_engine()

In [24]:
query = """SELECT homebranch,location,ccode,itemcallnumber,dateaccessioned,notforloan, onloan
FROM koha_prod.items i 
WHERE homebranch = 'MED'
AND notforloan = 0"""

In [25]:
docs_med = pd.read_sql(query,db_conn) 

In [26]:
query = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'collection'"""

va_collection = pd.read_sql(query, db_conn)

docs_med = docs_med.merge(va_collection,left_on="ccode",right_on="authorised_value",how="left")

docs_med = docs_med.rename(columns={"lib": "collection"})

docs_med = docs_med.drop(columns=["authorised_value"])

In [27]:
query = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'etat'"""

va_notforloan = pd.read_sql(query, db_conn)

va_notforloan['authorised_value'] = va_notforloan['authorised_value'].astype(int)

docs_med = docs_med.merge(va_notforloan,left_on="notforloan",right_on="authorised_value",how="left")

docs_med = docs_med.rename(columns={"authorised_value": "valeurs autorisées", "lib": "statut"})

docs_med = docs_med.drop(columns=['valeurs autorisées'])

In [28]:
total_docs_med = len(docs_med)

In [35]:
def docs_by_collection(df,location):
    df['nombre_docs'] = 1
    resultat = pd.pivot_table(df[df['location']==location],
                              index='collection',
                              values='nombre_docs',
                              aggfunc=sum,
                              sort=True
                             )    
    resultat['part_docs_med_en_%'] = round((resultat['nombre_docs']/total_docs_med)*100,2)
    resultat['part_docs_espace_en_%'] = round((resultat['nombre_docs']/resultat['nombre_docs'].sum())*100,1)
    resultat = resultat.sort_values(by='nombre_docs',ascending=False)
    
    return(resultat)

In [36]:
docs_med

,homebranch,location,ccode,itemcallnumber,dateaccessioned,notforloan,onloan,collection,statut,nombre_docs
0,MED,MED0A,JABEFAE,EXT E C/ALE,2005-03-22,0,None,JAB - Albums pour enfants,3 - Empruntable,1
1,MED,MED0A,JABEFAE,EXT E C/ALE,2005-03-22,0,2018-07-07,JAB - Albums pour enfants,3 - Empruntable,1
2,MED,MED0A,JABEFAE,EXT E C/ALE,2005-03-22,0,None,JAB - Albums pour enfants,3 - Empruntable,1
3,MED,MED0A,JABEFAE,EXT E C/ALE,2005-03-22,0,None,JAB - Albums pour enfants,3 - Empruntable,1
4,MED,MED0A,JABEFAE,EXT E C/ALE,2005-03-22,0,None,JAB - Albums pour enfants,3 - Empruntable,1
...,...,...,...,...,...,...,...,...,...,...
206002,MED,MED1A,AAPPLCB,Doc./641.1 ABC,2024-09-20,0,None,AAP - PL - Cuisine et bricolage,3 - Empruntable,1
206003,MED,MED1A,AAPPLMS,Doc./616.8 ALI,2024-09-20,0,2024-10-12,AAP - PL - Médecine et santé,3 - Empruntable,1
206004,MED,MED1A,AAPATAP,Doc./731 BRA,2024-09-20,0,None,AAP - Arts plastiques,3 - Empruntable,1
206005,MED,MED1A,AAPCIFI,DVD/F NOB,2024-09-20,0,2024-10-11,AAP - Films,3 - Empruntable,1


In [39]:
docs_med_rdc = docs_by_collection(df=docs_med,location='MED0C')
docs_med_lac = docs_by_collection(df=docs_med,location='MED1A')
docs_med_jeunesse = docs_by_collection(df=docs_med,location='MED1A')
docs_med_phare = docs_by_collection(df=docs_med,location='MED3A')
docs_med_collectivites = docs_by_collection(df=docs_med,location='MED0A')

In [43]:
docs_med_phare

,nombre_docs,part_docs_med_en_%,part_docs_espace_en_%
collection,,,
ACF - Histoire,3882,1.88,24.8
ACF - Sciences sociales,3724,1.81,23.8
ACF - Langues étrangères,2115,1.03,13.5
ACF - Sciences et technologies,1852,0.90,11.8
ACF - Religions,1315,0.64,8.4
ACF - Philosophie,1115,0.54,7.1
ACF - Réussir,782,0.38,5.0
ACF - Informatique,385,0.19,2.5
ACF - Français et FLE,369,0.18,2.4
